In [2]:
import argparse
import os
from matplotlib import pyplot as plt

from pyveg.src.data_analysis_utils import *
from pyveg.src.plotting import *

In [23]:
# results directory from `download_gee_data` script, containing `results_summary.json`
input_dir = '/Users/crangelsmith/PycharmProjects/GEE_DATA/TigerBush_Niger_2.59_13.12_2015-01-01_2020-01-01__2020-04-14_15-11-56/'
json_summary_path = os.path.join(input_dir, 'results_summary.json')

# put output plots in the results dir
output_dir = os.path.join(input_dir, 'analysis')

In [24]:
# read all json files in the directory and produce a dataframe
print(f"Reading results from '{os.path.abspath(json_summary_path)}'...")
dfs = variable_read_json_to_dataframe(json_summary_path)

print (dfs.keys())

Reading results from '/Users/crangelsmith/PycharmProjects/GEE_DATA/TigerBush_Niger_2.59_13.12_2015-01-01_2020-01-01__2020-04-14_15-11-56/results_summary.json'...
dict_keys(['COPERNICUS/S2', 'ECMWF/ERA5/MONTHLY'])


In [25]:
print (dfs['COPERNICUS/S2'].head())
print (dfs['ECMWF/ERA5/MONTHLY'].head())

         date                                        feature_vec  latitude  \
0  2015-07-15  [0.0, -31.0, -59.0, -71.0, -116.0, -130.0, -22...    2.6377   
1  2015-07-15  [0.0, -17.0, -69.0, -104.0, -169.0, -200.0, -2...    2.5514   
2  2015-07-15  [0.0, -49.0, -103.0, -171.0, -249.0, -292.0, -...    2.5741   
3  2015-07-15  [0.0, -13.0, -39.0, -62.0, -103.0, -132.0, -20...    2.5923   
4  2015-07-15  [0.0, -33.0, -81.0, -112.0, -164.0, -226.0, -2...    2.6286   

   longitude    mean  offset  offset50  slope         std  
0    13.1086 -456.70 -1082.0    -640.0 -54.10  343.613897  
1    13.1223 -459.10 -1026.0    -568.0 -51.30  318.552492  
2    13.1405 -621.65 -1325.0    -684.0 -66.25  406.338440  
3    13.1359 -398.95  -919.0    -527.0 -45.95  292.988818  
4    13.1495 -516.35 -1145.0    -617.0 -57.25  357.583176  
         date  mean_2m_air_temperature  total_precipitation
0  2015-01-16               296.405121             0.000000
1  2015-02-15               303.000031             

In [ ]:
  # spatial analysis and plotting 

# from the dataframe, produce network metric figure for each avalaible date
print('\nCreating spatial plots...')

# create new subdir for time series analysis
spatial_subdir = os.path.join(output_dir, 'spatial')
if not os.path.exists(spatial_subdir):
    os.makedirs(spatial_subdir, exist_ok=True)

for collection_name, df in dfs.items():
    if collection_name == 'COPERNICUS/S2' or 'LANDSAT' in collection_name:
        data_df_geo = convert_to_geopandas(df)
        data_df_geo_coarse = coarse_dataframe(data_df_geo, 2)
        create_lat_long_metric_figures(data_df_geo_coarse, 'offset50', spatial_subdir)



Creating spatial plots...
